# Experiment: Kwaliteitanalyse
**Beschrijving:** Het doel is om een dataframe op te zetten die laat zien wat de kwaliteit is van de data.

**Auteur:** ICTDSd1


## Doel en Verwachtingen
**Doelstelling:** We willen laten zien wat de kwaliteit van onze dataset is en dit in een dataframe neerzetten.

**Verwachting:** We verwachten dat de kwaliteit van de data uit de dataset zeer verschillend zal zijn.

## Data en Bronnen
**Datasets:**
- We gebruiken de fragments_data.csv die we gebruikt hebben voor de clustering van de geluidsfragmenten.


## Business Understanding
N.v.t.

## Data Understanding
De data set die we gebruiken hebben we zelf opgesplitst in fragmenten van 2 seconden. Deze dataset bevat 10 verschillende soorten vogelgeluiden. We zullen de data opsplitsen in train en test data. en om de experimenten snel uit te voeren maken we gebruik van de test data.

In [11]:
import pandas as pd
import numpy as np  
fragmenten_df = pd.read_csv('Data/fragments_data.csv')
kwaliteit_df = fragmenten_df.copy()
kwaliteit_df.head()

,fragment,labels,label_numeric
0,05190.100417.watersniproepEempolder_fragment_0...,watersnip,0
1,05190.100417.watersniproepEempolder_fragment_1...,watersnip,0
2,05190.100417.watersniproepEempolder_fragment_2...,watersnip,0
3,05190.100417.watersniproepEempolder_fragment_3...,watersnip,0
4,05190.100417.watersniproepEempolder_fragment_4...,watersnip,0


In [12]:
from sklearn.model_selection import train_test_split
train_data1, test_data1 = train_test_split(kwaliteit_df, test_size=0.3, random_state=42)
test_data1.head()

,fragment,labels,label_numeric
948,XC466471-Redshank (2)_fragment_0.wav,tureluur,1
3036,XC749107-2022-09-14 2020 Oeverlopers_fragment_...,oeverloper,4
1960,XC590410-200918_0480_2257_Watersnip_fragment_1...,watersnip,0
3543,XC834752-01.-1950-2050-watersnip_fragment_5.wav,watersnip,0
1689,"XC561755-Oeverloper, flight call, IGS, 050520,...",oeverloper,4


In [13]:
test_data1['labels'].value_counts()

labels
tureluur                  362
oeverloper                256
witgat                    202
watersnip                 170
zwarte ruiter             149
geelpootruiter             11
grauwe franjepoot          10
steenloper                  7
steltloper                  3
amerikaanse oeverloper      1
Name: count, dtype: int64

## Data Preparation


### SNR
De SNR (Signaal-Ruisverhouding) is de verhouding tussen het gewenste signaal en de achtergrondruis in een audiobestand. Het wordt gemeten in decibels (dB). Een SNR-waarde hoger dan 0 dB geeft aan dat er meer signaal is dan ruis, wat duidt op een goede geluidskwaliteit. Dit betekent dat het gewenste geluid duidelijk is ten opzichte van de storende ruis.

Om de SNR-waarde in een begrijpelijker context te plaatsen, hier zijn enkele algemene richtlijnen:

- 5 dB tot 10 dB: Dit wordt beschouwd als "Minimum" signaalniveau. Het ruisniveau is bijna niet te onderscheiden van het gewenste signaal (bruikbare informatie)
- 10 dB tot 15 dB: Dit wordt beschouwd als "Matig" signaalniveau.
- 15 dB tot 25 dB: Dit wordt beschouwd als "Acceptabel" signaalniveau.
- 25 dB tot 40 dB: Dit wordt beschouwd als "Goed" signaalniveau.
- 41 dB of hoger: Dit wordt beschouwd als "Uitstekend" en duidt op een zeer sterke en helder geluid.

In [14]:
import librosa
def calculate_snr(audio_path):
    audio_data, sample_rate = librosa.load(audio_path)
    signal_rms = np.sqrt(np.mean(audio_data**2))
    noise_rms = np.sqrt(np.mean(audio_data[0:sample_rate]**2))  # Bereken het ruisniveau in het eerste seconde van het audio
    snr = 20 * np.log10(signal_rms / noise_rms)
    return snr

In [15]:
def categorize_quality(snr):
    if snr < 5:
        return 'Ongeschikt'
    elif 5 <= snr < 10:
        return 'Minimum'
    elif 10 <= snr < 15:
        return 'Geaccepteerd minimum'
    elif 15 <= snr < 25:
        return 'Acceptabel'
    elif 25 <= snr < 41:
        return 'Goed'
    else:
        return 'Uitstekend'

In [ ]:

for fragment in test_data1['fragment']:
    y, sr = librosa.load("audio_fragments/" + fragment, sr=None)
    test_data1['SNR'] = test_data1['fragment'].apply(calculate_snr)
    

## Modeling
**Gebruikte Modellen:**
- Random Forest (n_estimators=100, max_depth=5)
- Support Vector Machine (kernel='rbf', C=1)

**Evaluatie:**
- Random Forest: Nauwkeurigheid = 82%, F1 = 0.74
- SVM: Nauwkeurigheid = 78%, F1 = 0.70

## Evaluation
**Resultaten:**
- Random Forest model presteerde het beste met een nauwkeurigheid van 82% en een F1-score van 0.74.
- Het model voldoet niet aan de minimale eisen van 85% nauwkeurigheid.

## Deployment
**Conclusie:**
- Het huidige model voldoet niet aan de eisen. Meer feature engineering is nodig.

**Volgende Stappen:**
- Meer geavanceerde features toevoegen uit interactiedata.
- Hyperparameter tuning uitvoeren.